In [5]:
import pandas as pd
import os
#import pyprind

In [5]:
!ls data/aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [6]:
!pwd data/aclImdb/test

/home/coolck/Development/ML/TensorFlow/Sentiment-Analysis-MovieData


In [6]:
basepath = 'data/aclImdb'

In [7]:
labels = {'pos' : 1, 'neg' : 0}

In [8]:
df = pd.DataFrame()

In [9]:
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file),
                      'r', encoding = 'utf-8') as infile:
                txt = infile.read()
            df = df.append([ [txt, labels[l]] ] ,
                           ignore_index = True
                          )
            

In [10]:
df.columns = ['review','sentiment']

In [11]:
df.head()

,review,sentiment
0,there's only so much that i can take of Filipi...,1
1,I happened to catch about the last 45 minutes ...,1
2,"Functioning as a sort of midpoint between ""Wai...",1
3,"The movie was very good when it came out, I at...",1
4,We just saw this movie in Austin Texas at the ...,1


In [12]:
import numpy as np

In [13]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index = False, encoding = 'utf-8')

In [14]:
df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head()

,review,sentiment
0,"This movie has to be seen for the music, which...",1
1,"Oh God, Why? I am aghast at the sheer ineptitu...",0
2,"I'm a big fan of horror flicks, and zombie fil...",0
3,I first saw this film when I was flipping thro...,1
4,What a horrible movie. I cannot believe i wast...,0


In [3]:
!ls 

data  DownloadData.ipynb  make_csv.ipynb  movie_data.csv


In [18]:
df.tail()

,review,sentiment
49995,"I watched this film when I was a kid, and I th...",0
49996,Pros: Nothing<br /><br />Cons: Everything<br /...,0
49997,I saw this on TV the other night or rather I ...,0
49998,Patrick Channing (Jeff Kober) is a disciple of...,0
49999,"One of quite a few cartoon Scooby Doo films, ""...",1


In [23]:
df.loc[0, 'review'][-999:]

" The cantata can be appreciated for itself, but having seen the movie helps.<br /><br />The story itself is fairly standard Soviet wartime anti-German propaganda. An amusing anecdote is that Stalin was later so eager to buy time by appeasing Hitler at the onset of World War II that Eisenstein was seconded to a German movie for a short while, as penance.<br /><br />Prince Alexander Nevsky (revered as a saint by the Russian Orthodox), having earned his nickname by defeating Swedish raiders on the river Neva, musters Russian resistance against an invasion by the Teutonic knights, and concludes his victory with a stern warning to other would-be invaders.<br /><br />An interesting scene: Alexander sees a convoy of Russians being taken as slaves by Mongolian soldiers, and does nothing to stop this, as the Teutonic knights are his priority. The historical Alexander Nevsky paid tribute to the Mongol khans, as his father before him, while working to consolidate his realm's future independence.

<h1>Bag Of Words</h1>
<h2>Creating feature vectors</h2>

In [24]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(
                [
                    'This sun is shining',
                    'The weather is sweet',
                    'The sun is shining and the weather is sweet'
                ]
                )
bag = count.fit_transform(docs)

In [26]:
print(count.vocabulary_)

{'this': 6, 'sun': 3, 'is': 1, 'shining': 2, 'the': 5, 'weather': 7, 'sweet': 4, 'and': 0}


In [27]:
print(bag.toarray())

[[0 1 1 1 0 0 1 0]
 [0 1 0 0 1 1 0 1]
 [1 2 1 1 1 2 0 1]]


<h2>tf-idf transform</h2>

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(
                            use_idf = True,
                            norm = 'l2' , #normalize,
                            smooth_idf = True
                        )
np.set_printoptions(precision = 2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.37 0.48 0.48 0.   0.   0.63 0.  ]
 [0.   0.41 0.   0.   0.53 0.53 0.   0.53]
 [0.38 0.45 0.29 0.29 0.29 0.57 0.   0.29]]


<h2>Clearning text-data</h2>
<ul>
    <li>Remove HTML</li>
    <li>Remove capitals</li>
    <li>Get emoticons</li>
</ul>

In [33]:
"""
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(D|P))', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.json(emoticons).replace('-', ''))
    return text
"""
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [34]:
preprocessor(df.loc[0, 'review'][-999:])

' the cantata can be appreciated for itself but having seen the movie helps the story itself is fairly standard soviet wartime anti german propaganda an amusing anecdote is that stalin was later so eager to buy time by appeasing hitler at the onset of world war ii that eisenstein was seconded to a german movie for a short while as penance prince alexander nevsky revered as a saint by the russian orthodox having earned his nickname by defeating swedish raiders on the river neva musters russian resistance against an invasion by the teutonic knights and concludes his victory with a stern warning to other would be invaders an interesting scene alexander sees a convoy of russians being taken as slaves by mongolian soldiers and does nothing to stop this as the teutonic knights are his priority the historical alexander nevsky paid tribute to the mongol khans as his father before him while working to consolidate his realm s future independence '

In [35]:
preprocessor("</a> This is :) :( a test :-) !")

' this is a test :) :( :)'

In [36]:
df['review'] = df['review'].apply(preprocessor)

<h3>Tokens | Word-Stemming</h3>

In [37]:
def tokenizer(text):
    return text.split()

In [38]:
tokenizer('running like running and thus they run')

['running', 'like', 'running', 'and', 'thus', 'they', 'run']

In [39]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [40]:
tokenizer_porter('running like running and thus they run')

['run', 'like', 'run', 'and', 'thu', 'they', 'run']

<h3>Stop word removal</h3>

In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/coolck/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
from nltk.corpus import stopwords 

stop = stopwords.words('english')
[w for w in tokenizer_porter('a running likes running and runs a lot')[-10:] if w not in stop]

['run', 'like', 'run', 'run', 'lot']

<h1>Document Classification</h1>

In [44]:
#Creatting training set and test set
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000: , 'review'].values
y_test = df.loc[25000: , 'sentiment'].values

<h3>GridSearchCV : find optimal set of parameters for logistic regression using 5-fold cross validation</h3>

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer 

In [46]:
tfidf = TfidfVectorizer(
                        strip_accents = None,
                        lowercase = False,
                        preprocessor = None
                        )

In [48]:
param_grid = [{
                    'vect_ngram_range' : [(1,1)],
                    'vect_stop_words' : [stop, None],
                    'vect_tokenizer' : [tokenizer, tokenizer_porter],
                    'clf_penalty' : ['l1', 'l2'],
                    'clf_C' : [1.0, 10.0, 100.0]
                },
                {
                    'vect_ngram_range' : [(1,1)],
                    'vect_stop_wods' : [stop, None],
                    'vect_tokenizer' : [tokenizer, tokenizer_porter],
                    'vect_use_idf' : [False],
                    'vect_norm' : [None],
                    'clf_penalty' : ['l1', 'l2'],
                    'cls_C' : [1.0, 10.0, 100.0]
                }
                ]

In [49]:
lr_tfidf = Pipeline([
                        ('vect', tfidf), ('clf', LogisticRegression(random_state = 0))
                    ])

In [53]:
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring = 'accuracy', cv = 5, verbose = 1, n_jobs = -1)

In [54]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/coolck/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/coolck/anaconda3/lib/py

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [56]:
if 'TRAVIS' in os.environ:
    gs_lr_tfidf.verbose=2
    X_train = df.loc[:250, 'review'].values
    y_train = df.loc[:250, 'sentiment'].values
    X_test = df.loc[25000:25250, 'review'].values
    y_test = df.loc[25000:25250, 'sentiment'].values

In [57]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 35.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 175.2min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 224.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's...se_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_tr

<h3>Choosing the best parameters</h3>

In [59]:
print("Best parameter set: %s "% gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7f2d4fcc3ea0>} 
